In [40]:
#Imports
from deap import creator, tools, base
import numpy as np
import timeit
from stage import fullPrint, getStage
from datetime import datetime, date, timedelta
from sklearn.base import BaseEstimator
import random
import pandas as pd
import yahoo_fin.stock_info as yf
from sklearn.model_selection import train_test_split
import pickle
# here
# from multiprocessing import Pool

In [41]:
start = timeit.default_timer()
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [42]:
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")

train, test = train_test_split(nasdaqList, test_size=0.3, shuffle=True)
# deleteAfterTest
train, test = train_test_split(nasdaqList[0:3000], test_size=0.3, shuffle=True)
#end
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2, trainSet3]
resultFile = open("resultML.txt","a")
resultFile.write("train "+str(train)+"\n")
resultFile.write("test "+str(test)+"\n")
resultFile.close()

now = datetime.now()
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")


In [43]:
removeList = []
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            removeList.append(symbol)
            print(symbol)
            continue
        if not (df['close'] != 0).all() or df.empty:
            if df.empty:
                print(symbol)
                removeList.append(symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = calculateGroupReturn(train)
for element in removeList:
    nasdaqList.remove(element)
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(nasdaqList, f)



In [45]:
indexNum = 0
sum = [1,1,1,1,1,1]
for i in range(6):
    if i < 3:
        l = trainSets[i]
    else:
        l = np.concatenate((trainSets[(i+1)%3],trainSets[(i+2)%3]))
    index = 0
    while index != len(l):
        if l[index] not in listOfDf.columns:
            l = np.delete(l, index)
        else:
            index += 1
    for index, element in listOfDf[l].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        if len(listOfStockRet) != 0:
            sum[i] = sum[i] * np.mean(listOfStockRet)     
for i in range(1,6):
    sum[i] = sum[i] / sum[0]
sum[0] = 1.0
print(sum)

[1.0, 0.8091820284251093, 0.394248099491251, 0.5562325631739403, 0.6265123582833875, 0.922702079209024]


In [46]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0,0,1,0.1,1,0.995,0.9,0.985])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.001))
  noises.append(np.random.normal(0, 0.1))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.004))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.0005))
noises.append(np.random.normal(0, 0.0025))

noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
noises.append(np.random.normal(0, 0.0025))
noises.append(np.random.normal(0, 0.0002))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [47]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [48]:
SECTORS = 6
def evalReturn(individual):
    print(individual[0])
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > individual[0][9] or df.at[date,'RS'] > individual[0][10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf, trainSets)
    scores = stockStageEstimator.score()
    result = stockStageEstimator.result()
    print(scores)
    print(np.average(scores))
    print(result)
    print(stockStageEstimator.getReturns())
    if scores == -1:
        return 0,
    if np.average(scores) >  0.75:
        return 0,
    paramFile = open("params.txt","a")
    paramFile.write(str(individual[0])+"\n")
    paramFile.write(str(result)+"\n")
    paramFile.close()
    resultFile = open("resultML.txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(np.average(scores))+"\n")
    resultFile.write(str(stockStageEstimator.getReturns())+"\n")
    resultFile.write(str(result)+"\n")
    resultFile.close()
    return result,

In [49]:
toolbox.register("evaluate", evalReturn)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [50]:
HOLDING = 0
TOTAL = 1
class StockStageEstimator(BaseEstimator):
    def __init__(self, paramList = [0.20, 1.39, 0.35, 0.93, 1.09, 1.22, 0.98, 0.98, 1.38, 0.99, -0.01, 0.97, 0.99, 0.96, 0.98], goodSectorDf = pd.DataFrame, sets = [[]]):
        self.paramList = paramList
        self.goodSectorDf = goodSectorDf
        self.returns = [0,0,0,0,0,0]
        self.sets = sets
        self.scores = [0,0,0]
    
    def evalFit(self, tickers, goodSectorDf):
        transactionFit = pd.read_pickle("transactionTemplate.pkl")
        transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
        #print("Reach #1")
        for symbol in tickers:
            df = getStage(symbol,self.paramList,goodSectorDf)
            inStage = False
            buyTwice = False
            if df.empty:
                continue
            for index, element in df.iterrows():
                # index = index.strftime('%Y-%m-%d')
                i = transactionFit.index.get_loc(index)
                if element.Stage == "Stage 2" or element.Stage == "Buy":
                    transactionFit.iat[i,HOLDING].append((symbol,element.close,0))
                    if buyTwice:
                        transactionFit.iat[i, HOLDING].append((symbol,element.close,0))
                    if element.Stage == "Buy" and inStage:
                        transactionFit.iat[i, HOLDING].append((symbol,element.close,0))
                        buyTwice = True
                    inStage = True
                    continue
                if "Sell" in element.Stage:
                    transactionFit.iat[i, HOLDING].append((symbol,element.close,-1))
                    if buyTwice:
                        transactionFit.iat[i, HOLDING].append((symbol,element.close,-1))
                    inStage = False
                    buyTwice = False
        #print("Reach #2")
        total = 100
        prevIndex = ''
        first = True
        transactionFitCopy = transactionFit
        transactionFitCopy['total'] = 0
        transactionFitCopy.to_csv("onlyShare.csv")
        #print("Reach #3")
        for index, element in transactionFitCopy.iterrows():
            indexNum = transactionFitCopy.index.get_loc(index)
            if first:
                first = False
                transactionFitCopy.iat[indexNum,1] = 100
                # print(i,transactionFitCopy.iat[i,1],"#1")
                continue
            if len(element.holding) == 0:
                transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                # print(i,transactionFitCopy.iat[i,1],"#2")
                continue
            else:
                prevData = transactionFitCopy.iat[indexNum-1, 0]
                if len(prevData):
                    total = 0
                    removeList = []
                    close = 0
                    for i in range(len(prevData)):
                        for j in range(len(element.holding)):
                            if element.holding[j][0] == prevData[i][0]:
                                close = element.holding[j][1]
                                if element.holding[j][2] == -1:
                                    removeList.append(element.holding[j])
                                break
                        total += close*prevData[i][2]
                    transactionFitCopy.iat[indexNum,1] = total
                    # print(i,transactionFitCopy.iat[indexNum,1],"#3")
                    for delete in removeList:
                        if delete in element.holding:
                            element.holding.remove(delete)
                else:
                    transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                    # print(i,transactionFitCopy.iat[i,1],"#4")
            if len(element.holding):
                allocation = total/len(element.holding)
            for i in range(len(element.holding)):
                element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
            prevIndex = index
        stockHolding  = 0
        transactionFitCopy.to_csv("estimatorTest.csv")
        for i in transactionFitCopy.iterrows():
            stockHolding += len(i[1]['holding'])
        if stockHolding/(transactionFitCopy.shape[0])<(len(tickers)/1000) or transactionFitCopy.iat[-1,TOTAL]<=100:
            print(stockHolding)
            print(transactionFitCopy.iloc[-1]['total'])
            print(transactionFitCopy)
            return -1
        return transactionFitCopy.iloc[-1]['total']

    def fit(self):
        for i in range(3):
            setsCombined = np.concatenate((self.sets[(i+1)%3],self.sets[(i+2)%3]))
            self.returns[i*2] = self.evalFit(setsCombined, self.goodSectorDf)
            if self.returns[i*2] == -1:
                return -1
            self.returns[i*2+1] = self.evalFit(self.sets[i%4], self.goodSectorDf)
            if self.returns[i*2+1] == -1:
                return -1
            
        return 0
    
    def score(self):
        if self.fit() == -1:
            return -1
        for i in range(3):
            self.scores[i] = np.absolute(1-((self.returns[i*2])/sum[i]/(self.returns[i*2+1])/sum[3+i]))
        return self.scores
    
    def result(self):
        return np.average(self.returns)
    
    def getReturns(self):
        return self.returns

In [51]:
def main():
    pop = toolbox.population(n=128)
    # Evaluate the entire population
    # here
    # pool = Pool()
    # tempResult = pool.map(toolbox.evaluate, pop)
    tempResult = map(toolbox.evaluate, pop)
    print(tempResult)
    fitnesses = list(tempResult)
    # here
    # pool.close()
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        if ind.fitness.values[0]<=100:
            del ind.fitness.values
    badInd = [ind for ind in pop if not ind.fitness.valid]
    while len(badInd)!=0:
        for ind in badInd:
            index = pop.index(ind)
            temp = toolbox.individual()
            pop[index] = temp
            del pop[index].fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, badInd)
        for ind, fit in zip(badInd, fitnesses):
            ind.fitness.values = fit
            if ind.fitness.values[0]<=100:
                del ind.fitness.values
        badInd = [ind for ind in pop if not ind.fitness.valid]
    
        
    # Begin the evolution
    for g in range(7):
        np.random.shuffle(train)
        trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
        trainSets = [trainSet1, trainSet2, trainSet3]
        print("-- Generation %i --" % g)
        resultFile = open("resultML.txt","a")
        resultFile.write("-- Generation %i --" % g+"\n")
        resultFile.close()
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        # here
        # pool = Pool()
        # fitnesses = pool.map(toolbox.evaluate, invalid_ind)
        # pool.close()
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
main()

[0.7903655305562709, 0.831251140149632, -0.08682921346882885, 1.0099440060521168, 1.3942464473157052, 1.135280264961849, 0.8200335004848869, 0.8589957961607246, 1.4090867751826246, 1.0024066811886827, 0.24574788949351237, 1.026639046568078, 0.9905823273486183, 0.8831339017093208, 0.984345513541798]
8725
0
           holding  total
1995-01-06      []    100
1995-01-13      []    100
1995-01-20      []    100
1995-01-27      []    100
1995-02-03      []    100
...            ...    ...
2022-01-28      []      0
2022-02-04      []      0
2022-02-11      []      0
2022-02-18      []      0
2022-02-25      []      0

[1368 rows x 2 columns]
-1
-1.0
-0.16666666666666666
[-1, 0, 0, 0, 0, 0]
[0.42187041287639004, 1.7768394231028632, 0.2757731652590345, 1.0353939129711849, 1.2988340300264967, 1.267704243771249, 0.8992523645822765, 1.0909381478567561, 1.281285579213192, 0.9999884118183473, 0.17483812125248105, 1.0246452291892498, 0.9956518466718152, 0.8901647378958923, 0.9865773293823726]
6930
0

KeyboardInterrupt: 

In [52]:
evalReturn([[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804]])

[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804]
[0.5759774333442693, 15.59985087619421, 10.809843520811551]
8.99522394345001
9527.981382385135
[4187.054126442339, 17752.650154091123, 20123.356973486476, 2391.2246220351976, 10313.046956876258, 2400.5554613794125]


(0,)

In [27]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
    goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    index = 0
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                # print(individual[0][9],individual[0][10])
                if  df.at[date,'MA30Slope'] > 0 or df.at[date,'RS'] > 0.1:
                    # if df.at[date,'close'] > df.at[date,'MA30']:
                        # print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
            except:
                listOfSector.append(df.iat[-1,6]) 
                pass
            # print(listOfSector)
        goodSectorDf.iat[index,0] = set(listOfSector)
        index = index + 1
df = getStage("AACG",[0.9630183179676801, 1.3518854933368092, 0.16319993430933552, 1.0537971763840923, 1.006835620568861, 1.0103606174928899, 0.9033455933504492, 0.9731546594459972, 1.46426124905874, 0.9991612063815158, 0.09863994265206354, 0.9826811057391516, 0.9949684774619274, 0.8670538938494661, 0.9895837713208671],goodSectorDf)

            close    volume ticker  fiveYearHigh      WMA30  WMA30Slope  \
2008-02-01   9.51     28200   AACG          0.00        NaN    0.000000   
2008-02-08   9.01      7900   AACG          0.00        NaN    0.000000   
2008-02-15   9.00     10000   AACG          0.00        NaN    0.000000   
2008-02-22   9.62      3200   AACG          0.00        NaN    0.000000   
2008-02-29  10.30     12100   AACG          0.00        NaN    0.000000   
2008-03-07  10.84     10300   AACG          0.00        NaN    0.000000   
2008-03-14  11.80      7600   AACG          0.00        NaN    0.000000   
2008-03-28   8.71     80000   AACG          0.00        NaN    0.000000   
2008-04-04  11.02     35100   AACG          0.00        NaN    0.000000   
2008-04-11   9.78      4400   AACG          0.00        NaN    0.000000   
2008-04-18  10.60      4900   AACG          0.00        NaN    0.000000   
2008-04-25  10.50      4400   AACG          0.00        NaN    0.000000   
2008-05-02  10.60      56

In [28]:
fullPrint(df)

            close                                              Stage
2008-02-01   9.51                                                   
2008-02-08   9.01      volume -0.7198581560283688 1.3518854933368092
2008-02-15   9.00      volume 0.26582278481012667 1.3518854933368092
2008-02-22   9.62      volume -0.6799999999999999 1.3518854933368092
2008-02-29  10.30         RS -0.9265646160947767 0.16319993430933552
2008-03-07  10.84     volume -0.14876033057851235 1.3518854933368092
2008-03-14  11.80      volume -0.2621359223300971 1.3518854933368092
2008-03-28   8.71         RS -0.9074634151757225 0.16319993430933552
2008-04-04  11.02                 volume -0.56125 1.3518854933368092
2008-04-11   9.78      volume -0.8746438746438746 1.3518854933368092
2008-04-18  10.60      volume 0.11363636363636354 1.3518854933368092
2008-04-25  10.50     volume -0.10204081632653061 1.3518854933368092
2008-05-02  10.60       volume 0.2727272727272727 1.3518854933368092
2008-05-09  10.90       volume 1.0

In [7]:
fullPrint(pd.read_pickle("stockData/sectorCharts/XLC.pkl"))

                 open       high        low      close   adjclose    volume  \
2018-06-22  50.590000  50.610001  50.189999  50.490002  48.974186    181500   
2018-06-29  49.799999  49.840000  49.419998  49.529999  48.043007    152300   
2018-07-06  50.139999  50.816002  49.930000  50.770000  49.245777    199800   
2018-07-13  51.779999  51.799999  51.449001  51.689999  50.138157     80100   
2018-07-20  51.049999  51.186001  50.840000  50.869999  49.342781    158400   
2018-07-27  49.770000  49.830002  48.638000  48.910000  47.441624   1111200   
2018-08-03  48.090000  48.930000  48.090000  48.910000  47.441624    117200   
2018-08-10  49.529999  49.529999  49.150002  49.270000  47.790810    177600   
2018-08-17  48.660000  48.810001  48.400002  48.610001  47.150623    226900   
2018-08-24  48.959999  49.439999  48.880001  49.439999  47.955708    330900   
2018-08-31  49.619999  49.619999  49.042000  49.180000  47.703522    235000   
2018-09-07  47.279999  47.950001  47.230000  47.6800

In [26]:
getStage("AACG",)

            close    volume ticker  fiveYearHigh      WMA30  WMA30Slope  \
2008-02-01   9.51     28200   AACG          0.00        NaN    0.000000   
2008-02-08   9.01      7900   AACG          0.00        NaN    0.000000   
2008-02-15   9.00     10000   AACG          0.00        NaN    0.000000   
2008-02-22   9.62      3200   AACG          0.00        NaN    0.000000   
2008-02-29  10.30     12100   AACG          0.00        NaN    0.000000   
2008-03-07  10.84     10300   AACG          0.00        NaN    0.000000   
2008-03-14  11.80      7600   AACG          0.00        NaN    0.000000   
2008-03-28   8.71     80000   AACG          0.00        NaN    0.000000   
2008-04-04  11.02     35100   AACG          0.00        NaN    0.000000   
2008-04-11   9.78      4400   AACG          0.00        NaN    0.000000   
2008-04-18  10.60      4900   AACG          0.00        NaN    0.000000   
2008-04-25  10.50      4400   AACG          0.00        NaN    0.000000   
2008-05-02  10.60      56